# Basics: Defining Custom Games with `shapiq`

This notebook demonstrates how to define custom cooperative games and use them with the `shapiq` library.

In [1]:
import shapiq
import numpy as np

shapiq.__version__

'1.1.1'



## Introduction to Cooperative Game Theory and Shapley Values
Cooperative game theory deals with the study of games in which players/participants can form groups (also known coalitions) to achieve a collective payoff. More formally a cooperative game is defined as a tuple $(N,\nu)$ where:
- $N$ is a finite set of players
- $\nu$ is a characteristic function that maps every coalition of players to a real number, i.e. $\nu:2^N \rightarrow \mathbb{R}$

## Defining a Custom Cooperative Game
To illustrate the concept of cooperative games, we consider a simple example of a _cooking game_ you might find in a restaurant.
The game consists of three cooks, _Alice_, _Bob_, and _Charlie_, who are preparing a meal _together_.

The characteristic function $\nu$ maps each coalition of players to the quality of the meal:

| Coalition             | Quality |
|-----------------------|---------|
| {no cook}             | 0       |
| {Alice}               | 4       |
| {Bob}                 | 3       |
| {Charlie}             | 2       |
| {Alice, Bob}          | 9       |
| {Alice, Charlie}      | 8       |
| {Bob, Charlie}        | 7       |
| {Alice, Bob, Charlie} | 15      |

For example, the coalition {Alice, Bob} has a quality of 7, while the coalition {Alice, Bob, Charlie} has a quality of 15.
If no cooks participate, the quality of the meal is 0 and no meal is prepared.

We can easily model this general form of a cooperative game with `shapiq` by defining a class that inherits from the `shapiq.Game` class.
Note, a game does not necessarily have to be a subclass of `shapiq.Game` and can also be a simple function that defines the value function $\nu:2^N \rightarrow \mathbb{R}$.
Methods in `shapiq` can also be used with such functions. However, using the `Game` class provides a more structured way to define the game and its properties.
It also comes equipped with handy helper methods.

Below we define the `CookingGame` class that models the cooking game.

In [2]:
# define the cooking game as a subclass of shapiq.Game
class CookingGame(shapiq.Game):
    def __init__(self):
        self.characteristic_function = {
            (): 0,
            (0,): 4,
            (1,): 3,
            (2,): 2,
            (0, 1): 9,
            (0, 2): 8,
            (1, 2): 7,
            (0, 1, 2): 15,
        }
        super().__init__(
            n_players=3,
            player_names=["Alice", "Bob", "Charlie"],  # Optional list of names
            normalization_value=self.characteristic_function[()],  # 0
        )

    def value_function(self, coalitions: np.ndarray) -> np.ndarray:
        """Defines the worth of a coalition as a lookup in the characteristic function."""
        output = []
        for coalition in coalitions:
            output.append(self.characteristic_function[tuple(np.where(coalition)[0])])
        return np.array(output)


cooking_game = CookingGame()
cooking_game

CookingGame(3 players, normalize=False, normalization_value=0, precomputed=False)

### Querying the Value Function of the Game
We can query the value function of the game for different coalitions.

In [3]:
# query the value function of the game for different coalitions
coals = np.array([[0, 0, 0], [1, 1, 0], [1, 0, 1], [0, 1, 1], [1, 1, 1]])
cooking_game(coals)

array([ 0,  9,  8,  7, 15])

In [4]:
# query the value function with the names of the players
coals = [
    (),
    ("Alice", "Bob"),
    ("Alice", "Charlie"),
    ("Bob", "Charlie"),
    ("Alice", "Bob", "Charlie"),
]
cooking_game(coals)

array([ 0,  9,  8,  7, 15])

In [5]:
# we can automatically get the value of the grand coalition
print("The quality of the meal for the grand coalition is:", cooking_game.grand_coalition_value)

# similarly we can get the value of the empty coalition
print("The quality of the meal for the empty coalition is:", cooking_game.empty_coalition_value)

The quality of the meal for the grand coalition is: 15.0
The quality of the meal for the empty coalition is: 0.0


### Precomputing game values
If our game is small (low number of players), we can precompute the values of all coalitions.
This is useful for games where the value function is computationally expensive and we might want to use the values multiple times.

If a game is defined as a subclass of `shapiq.Game`, we can use the `precompute` method to precompute the values of all coalitions.

In [6]:
# see that no values have been precomputed:
print("Coalitions stored before precomputation:", cooking_game.coalition_lookup)
print("Values stored before precomputation:    ", cooking_game.value_storage)

# precompute the values of all coalitions
cooking_game.verbose = True  # to see progress
cooking_game.precompute()

# see that all values have been precomputed:
print("Coalitions stored after precomputation:", cooking_game.coalition_lookup)
print("Values stored after precomputation:    ", cooking_game.value_storage)

Coalitions stored before precomputation: {}
Values stored before precomputation:     []


Evaluating game:   0%|          | 0/8 [00:00<?, ? coalition/s]

Coalitions stored after precomputation: {(): 0, (0,): 1, (1,): 2, (2,): 3, (0, 1): 4, (0, 2): 5, (1, 2): 6, (0, 1, 2): 7}
Values stored after precomputation:     [ 0.  4.  3.  2.  9.  8.  7. 15.]


The precomputed values are stored in the `coalition_lookup` and `value_storage` attributes of the game object.
You can even save the precomputed values to a file and load them later using the `save_values` and `load_values` methods.

In [7]:
# save the precomputed values to a file
cooking_game.save_values("data/cooking_game_values.npz")

# load the precomputed values from the file
empty_cooking_game = CookingGame()
print("Values stored before loading: ", empty_cooking_game.value_storage)
empty_cooking_game.load_values("cooking_game_values.npz")
print("Values stored after loading:  ", empty_cooking_game.value_storage)

Values stored before loading:  []


FileNotFoundError: [Errno 2] No such file or directory: 'cooking_game_values.npz'

## Initializing Games Directly from Precomputed Values
If we have precomputed values for a game, we can directly initialize a game object from these values.

In [ ]:
# initialize a game object directly from precomputed values
game = shapiq.Game(path_to_values="data/cooking_game_values.npz")
print(game)

# query the value function of the game for the same coalitions as before
coals = np.array([[0, 0, 0], [1, 1, 0], [1, 0, 1], [0, 1, 1], [1, 1, 1]])
game(coals)

Now we have a game object initialized directly from precomputed values.
Note that unlike our special `CookingGame` class, this game object does not certain attributes like the `characteristic_function`.

In [ ]:
print(cooking_game.characteristic_function)
try:
    print(game.characteristic_function)  # this line should throw an error
except AttributeError as e:
    print("AttributeError:", e)